In [1]:
%cd ..

/home/juanhevia/IDIL


In [2]:
# import idil_gym

In [2]:
# import os
# os.environ["LD_LIBRARY_PATH"] = os.environ.get("LD_LIBRARY_PATH", "") + ":/home/juanhevia/.mujoco/mujoco210/bin" + ":/usr/lib/nvidia"

#import gymnasium
# import gym
import json 
import pickle
import numpy as np 
import pandas as pd 
#import minari
# can we use the vector distance between the achieved and desired goal as a termination mark?
import matplotlib.pyplot as plt 

# Check proportion of macro goal sequences

In [3]:
# load train sequences
with open("notebooks/kitchen_franka_demos/FrankaKitchen-v0-mixed_260.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
from collections import Counter

rows = []

for traj in data["latents"]:
    _count = Counter(traj)
    # build JSON object as row
    rows.append(_count)

# convert to DataFrame
df = pd.DataFrame(rows)

In [5]:
df.isnull().any()

1    True
4    True
2    True
0    True
dtype: bool

In [6]:
unique_combs = []
for latent in data["latents"]:
    # compute the unique latents in the sequence without losing their order of appearance
    _unique = []
    for l in latent:
        if l not in _unique:
            _unique.append(l)
    
    unique_combs.append(tuple(_unique))

In [7]:
Counter(unique_combs)

Counter({(1, 4, 2): 68,
         (0, 4, 2): 86,
         (4, 2): 1,
         (0, 1, 4): 44,
         (0, 1, 2): 61})

In [8]:
sum(Counter(unique_combs).values())

260

## Split `pkl` files by macro goal sequence

In [9]:
available_seqs = list(Counter(unique_combs).keys())
# keep only 3-goals sequences
available_seqs = [seq for seq in available_seqs if len(seq) == 3]
available_seqs

[(1, 4, 2), (0, 4, 2), (0, 1, 4), (0, 1, 2)]

In [10]:
from collections import defaultdict

seqs_trajs = defaultdict(list)

# create a dict to hold trajectories indices for each sequence
for traj_idx, traj in enumerate(data["latents"]):
    # compute the unique latents in the sequence without losing their order of appearance
    _unique = []
    for l in traj:
        if l not in _unique:
            _unique.append(l)
            
    if tuple(_unique) in available_seqs:
        seqs_trajs[tuple(_unique)].append(traj_idx)
    
seqs_trajs.keys()

dict_keys([(1, 4, 2), (0, 4, 2), (0, 1, 4), (0, 1, 2)])

In [11]:
# check that lengths are the same
for seq, trajs in seqs_trajs.items():
    print(seq, " --> # of records = ", len(trajs))

(1, 4, 2)  --> # of records =  68
(0, 4, 2)  --> # of records =  86
(0, 1, 4)  --> # of records =  44
(0, 1, 2)  --> # of records =  61


In [12]:
# create new dictionaries to hold the sequences and the corresponding trajectories
split_data = {}

for seq, traj_indices in seqs_trajs.items():
    split_data[seq] = defaultdict(list)

    for key in data.keys():
        for idx in traj_indices:
            split_data[seq][key].append(data[key][idx])

In [13]:
# compute lengths of the sequences in the split data
for seq, data in split_data.items():
    print(seq, " --> keys : ", data.keys())
    print(seq, " --> # of records = ", len(data["latents"]))

    # ensure all keys in the data have equal length
    for key in data.keys():
        assert len(data[key]) == len(data["latents"])

(1, 4, 2)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(1, 4, 2)  --> # of records =  68
(0, 4, 2)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(0, 4, 2)  --> # of records =  86
(0, 1, 4)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(0, 1, 4)  --> # of records =  44
(0, 1, 2)  --> keys :  dict_keys(['states', 'next_states', 'actions', 'rewards', 'latents', 'lengths', 'dones'])
(0, 1, 2)  --> # of records =  61


In [14]:
print(len(split_data[(0, 1, 2)]["states"][0]))
print(len(split_data[(0, 1, 2)]["next_states"][0]))
print(split_data[(0, 1, 2)]["lengths"][0])

176
176
176


In [15]:
# save the split data to different pickle files
for seq, data in split_data.items():
    seq_label = "_".join([str(s) for s in seq])
    # with open(f"notebooks/kitchen_franka_demos/FrankaKitchen-v0-mixed_260_{seq_label}.pkl", "wb") as f:
    with open(f"idil_train/experts//FrankaKitchen-v0-mixed_260_{seq_label}.pkl", "wb") as f:
        pickle.dump(data, f)